In [1]:
import numpy as np

class Mancala:
    def __init__(self, holes=7, stones=7, board=None):
        self.n_holes = holes
        self.n_stones = stones
        self.north_store = self.n_holes
        self.south_store = self.n_holes * 2 + 1
        self.north = 'north'
        self.south = 'south'
        self.first_play = None

        self.reset()

        if board is not None:
            self.board = board
    
    # make a board of initial state
    def reset(self):
        self.board = np.full((self.n_holes + 1) * 2, self.n_stones)
        self.board[self.n_holes] = 0
        self.board[-1] = 0
        self.first_play = True

    def _post_step(self, side, last_pos):
        # if last position is in the player's scoring well and is not first play
        # instruct an extra move
        if self.is_store(side, last_pos) and not self.first_play:
            return f'{side}:continue'

        # if the last position is in an self empty hole while the according opponent's hole
        # has stones in it, take the stones from both holes
        if self.is_hole(side, last_pos) and self.board[last_pos] == 1 \
            and self.board[self.get_opponent_pos(last_pos)] != 0:
            scores_stored_pos = self.get_store(side)
            self.board[scores_stored_pos] = self.board[scores_stored_pos]+ self.board[last_pos] \
                + self.board[self.get_opponent_pos(last_pos)]
            self.board[last_pos] = 0
            self.board[self.get_opponent_pos(last_pos)] = 0


        # record first play
        if self.first_play:
            self.first_play = False

        # if the player has won, print out the message
        if self.has_over_half_stones(side):
            return f'{side}:has won'
        else:
            return self.get_opponent_side(side)

    # select a hole to move by player
    def step(self, side, hole):
        if not self.has_legal_move(side):
            return self.check_win()
        
        pos = self._get_board_pos(side, hole)
        stones = self.board[pos]
        self.board[pos] = 0
        pos += 1
        # record the position where last stone is placed
        last_pos = pos
        while stones > 0:
            if not self.is_opponent_store(side, pos):
                self.board[pos] += 1
                stones -= 1
                if stones == 0:
                    last_pos = pos
            # add the position
            # if position outside the board make it start from the beginning of the board by %
            pos = (pos + 1) % len(self.board)
        
        return self._post_step(side, last_pos)
        
    
    # return the position of player's scoring well
    def get_store(self, side):
        return self.south_store if side == self.south else self.north_store
            
    # swap the player after each normal round
    def get_opponent_side(self, side):
        return self.north if side == self.south else self.south
    
    # verify whether it is player's hole
    def is_hole(self, side, pos):
        return side == self.south and self.north_store < pos < self.south_store \
            or side == self.north and pos < self.south_store
    
    # get the according opponent hole position given the self hole position
    def get_opponent_pos(self, self_pos):
        assert self.is_hole(self.south, self_pos) or self.is_hole(self.north, self_pos)
        return 2 * self.n_holes - self_pos
        
    # verify whether is player's scoring well
    def is_store(self, side, pos):
        return side == self.south and pos == self.south_store \
            or side == self.north and pos == self.north_store
    
    # verify whether is opponent's scoring well
    def is_opponent_store(self, side, pos):
        return side == self.south and pos == self.north_store \
            or side == self.north and pos == self.south_store
    
    # get the position to move in the board array
    # assuming the side chosen is reasonable
    def _get_board_pos(self, side, hole):
        return hole - 1 if side == self.north else hole + self.n_holes
    
    # swap the side for north according to pie rule
    # assuming the swap instruction is reasonable
    def swap_side(self):
        self.board = np.roll(self.board, self.n_holes + 1)
        return "swapped:south"
        # board_copy = deepcopy(self.board)
        # for i in range(len(self.board)):
        #     board_copy[i] = self.board[(i+self.n_holes+1)%len(self.board)]
        # self.board = board_copy
    
    # check whether the player has won
    def has_over_half_stones(self, side):
        return self.board[self.get_store(side)] > 49

    def get_start_hole(self, side):
        return 0 if side == self.north else self.n_holes + 1
    
    # check whether the player has legal move
    def has_legal_move(self, side):
        start = self.get_start_hole(side)
        return np.any(self.board[start:start + self.n_holes])
        # for i in range (self.n_holes):
        #     if self.board[i+offset] > 0:
        #         return True
        # return False

    # when one player does not have legal move
    # check the board to see who has won
    def check_win(self):
        north_total = self.board[self.get_store(self.north)]
        south_total = self.board[self.get_store(self.south)]
        offset = self.n_holes+1
        for i in range (self.n_holes):
            north_total += self.board[i]
            self.board[i] = 0
        self.board[self.get_store(self.north)] = north_total
        for i in range (self.n_holes):
            south_total += self.board[i+offset]
            self.board[i+offset] = 0
        self.board[self.get_store(self.south)] = south_total
        if north_total == south_total:
            return "draw"
        elif north_total > south_total:
            return "north:has won"
        else:
            return "south:has won" 
        
    # print out of the board
    def __str__(self):
        formatter = {'int': lambda x: f'{x: >3d}'}
        return '{:2d}  {}\n    {}  {}'.format(
            self.board[self.north_store],
            np.array2string(self.board[0:self.north_store][::-1], formatter=formatter),
            np.array2string(self.board[self.north_store+1:self.south_store], formatter=formatter),
            self.board[self.south_store]
        )

In [2]:
# test for the board
def boardTest(board,board_to_compare):
    return np.array_equal(board,board_to_compare)

# test for the msg
def msgTest(msg,msg_to_compare):
    return msg == msg_to_compare

# test for the game board and output msg
def gameTest(msg,msg_to_compare,game,board_to_compare):
    return boardTest(game.board,board_to_compare) and msgTest(msg, msg_to_compare)


game = Mancala()

# Test the start of the board
print("Start of the game board is correct: ", end = " ")
print(boardTest(game.board,np.array([7,7,7,7,7,7,7,0,7,7,7,7,7,7,7,0])))

# Test a start move and the pie rule for no extra move
print("The game performs correctly for a start move: ", end = " ")
print(gameTest(game.step('south', 1), "north", game, np.array([7,7,7,7,7,7,7,0,0,8,8,8,8,8,8,1])))

# Test the swap case of North
print("The game performs correctly for a swap move: ", end = " ")
print(gameTest(game.swap_side(), "swapped:south", game, np.array([0,8,8,8,8,8,8,1,7,7,7,7,7,7,7,0])))

# Test a normal move
print("The game performs correctly for a normal move: ", end = " ")
print(gameTest(game.step('south', 2), "north", game, np.array([1,8,8,8,8,8,8,1,7,0,8,8,8,8,8,1])))

# Test a move which will lead to an extra move
game.board = np.array([0,8,8,8,8,8,8,1,7,7,7,7,7,7,7,0])
print("The game performs correctly for the extra move case: ", end = " ")
print(gameTest(game.step('south', 1), "south:continue", game, np.array([0,8,8,8,8,8,8,1,0,8,8,8,8,8,8,1])))


# Test a move which will take the stones from opponent
game.board = np.array([1,0,9,9,9,9,9,2 ,1,0,11,9,9,9,9,2])
print("The game performs correctly when need to take over opponet's stones: ", end = " ")
print(gameTest(game.step('north', 1), "south", game, np.array([0,0,9,9,9,9,9,12,1,0,11,9,9,0,9,2])))

# Test a move which passees two scoring wells
game.board = np.array([0,0,9,9,9,0,9,12,0,0,11,9,9,0,9,12])
print("The game performs correctly when passing both scoring wells: ", end = " ")
print(gameTest(game.step('north', 7), "south", game, np.array([0,0,9,9,9,0,0,24,1,1,12,10,10,1,0,12])))


# Test the case the game not end if one player has no legal move but it's his/her opponent's turn
game.board = np.array([0,0,0,0,0,0,0,42,0,0,1,16,2,2,1,34])
print("The game not end when the player has no legal move but is opponent's turn: ", end = " ")
print(gameTest(game.step('south', 3), "north", game, np.array([0,0,0,0,0,0,0,42,0,0,0,17,2,2,1,34])))

# Test the case the game ends if it's one player's turn and he/she has no legal move
game.board = np.array([0,0,0,0,0,0,0,42,0,0,1,16,2,2,1,34])
print("The game ends when it's a player's turn and no legal move left: ", end = " ")
print(gameTest(game.step('north', 3), "south:has won", game, np.array([0,0,0,0,0,0,0,42,0,0,0,0,0,0,0,56])))


# Test the case the game ends when both player has 49 stones
game.board = np.array([0,0,0,0,0,0,0,49,0,0,0,0,0,0,0,49])
print("The game ends with a draw when both players have 49 stones: ", end = " ")
print(gameTest(game.step('south', 7), "draw", game, np.array([0,0,0,0,0,0,0,49,0,0,0,0,0,0,0,49])))

# Test the case that the game not end when one player has 49 stones
game.board = np.array([7,4,0,2,2,0,1,48,2,6,1,1,2,3,0,19])
print("The game not end when one player has 49 stones: ", end = " ")
print(gameTest(game.step('north', 7), "north:continue", game, np.array([7,4,0,2,2,0,0,49,2,6,1,1,2,3,0,19])))

# Test the case that the game ends when one player has over 49 stones
print("The game ends when one player has over 49 stones: ", end = " ")
print(gameTest(game.step('north', 5), "north:has won", game, np.array([7,4,0,2,0,1,0,52,0,6,1,1,2,3,0,19])))

Start of the game board is correct:  True
The game performs correctly for a start move:  True
The game performs correctly for a swap move:  True
The game performs correctly for a normal move:  True
The game performs correctly for the extra move case:  True
The game performs correctly when need to take over opponet's stones:  True
The game performs correctly when passing both scoring wells:  True
The game not end when the player has no legal move but is opponent's turn:  True
The game ends when it's a player's turn and no legal move left:  True
The game ends with a draw when both players have 49 stones:  True
The game not end when one player has 49 stones:  True
The game ends when one player has over 49 stones:  True


### Random Agent
This is a random agent which moves randomly according to the position available

In [3]:
import random
import numpy as np
from copy import deepcopy

class RandomAgent:
    # return a move
    def get_move(board, side):
        if side == 'north':
            offset = 1
        else:
            offset = -7
        # find available moves
        available_moves = RandomAgent.get_available_move(board, side)
        # get a random move
        move = RandomAgent.get_random_move(available_moves)
        # convert the move from board index to move
        move += offset
        print("Random move is: " + str(move))
        return move

    # get the valid moves in the board
    # return the array of available move positions(in index) in board
    def get_available_move(board,side):
        valid_moves = []
        if side == 'north':
            offset = 0
        else:
            offset = 8
        # as long as the hole is empty, it is an available move
        for i in range (7):
            if (board[i+offset] != 0):
                valid_moves.append(i+offset)
        return valid_moves

    # get a random move from the available moves
    def get_random_move(available_moves):
        return random.choice(available_moves)

In [4]:
game = Mancala()
myboard = np.array([0,8,8,8,8,8,8,1,7,7,7,7,7,7,7,0])
game.board = myboard
player = 'south'
print(game)
game.step(player, RandomAgent.get_move(myboard, player))
print(game)

 1  [  8   8   8   8   8   8   0]
    [  7   7   7   7   7   7   7]  0
Random move is: 5
 1  [  8   8   8   9   9   9   1]
    [  7   7   7   7   0   8   8]  1


### Simple Agent
This is a simple agent which performs simple rules according to the available positions

#### Rules performed
- If one move can lead to win, perform that move
- If not record the position_score(position marks) of each position and perform the move which has the highest position_score. The critiria of the position marks is illustrated below:
  - If one move can lead to an extra move, it will gain the position mark according to how close it is to the scoring well. For example, move in position 7 can lead to extra move, it gains 7 position marks.
  - The increase in stones in the self scoring well is another way to gain position marks, if a move can increase n stones in the scoring well, it gets n position marks.
  - The third thing to take into account is the following opponent's move, in this case, we just assume the opponent's has a normal move(will not lead to extra moves). If in the following opponent's move can gain x stones, it get position marks -x.

In [5]:
import random
import numpy as np
from copy import deepcopy

class SimpleAgent:
    # return a move
    def get_move(board, side):
        if side == 'north':
            offset = 1
        else:
            offset = -7
        # find available moves
        available_moves = SimpleAgent.get_available_move(board, side)
        # get the suggested move
        move = SimpleAgent.get_suggested_move(board,side,available_moves)
        # convert the move from board index to move
        move += offset
        print("Suggest move is: " + str(move))
        return move

    # get the valid moves in the board
    # return the array of available move positions(in index) in board
    def get_available_move(board,side):
        valid_moves = []
        if side == 'north':
            offset = 0
        else:
            offset = 8
        # as long as the hole is empty, it is an available move
        for i in range (7):
            if (board[i+offset] != 0):
                valid_moves.append(i+offset)
        return valid_moves

    # check the increase in scores after a move
    def points_increase(old_board,new_board,side):
        if side == 'north':
            scoring_pos = 7
        else:
            scoring_pos = 15
        return new_board[scoring_pos] - old_board[scoring_pos]

    # check the max the player can got in the coming move
    # do not take into account extra moving case
    # in the SimpleAgent, it calculates the opponent's max increase
    def max_increase_normal(board,side,available_moves):
        # list to store all the increase in scores for each
        # avaliable moves
        scores_increase = []
        game = Mancala()
        # do a move to avoid pie rule
        game.step('north',1)
        if side == 'north':
            offset = 1
        else:
            offset = -7
        # perform moves for available moves
        for i in available_moves:
            newboard = deepcopy(board)
            game.board = newboard
            game.step(side, i+offset)
            scores_increase.append(SimpleAgent.points_increase(board,game.board,side))
        return max(scores_increase)

    # suggests a move using simple rules
    def get_suggested_move(board,side,available_moves):
        game = Mancala()
        # do a move to avoid pie rule
        game.step('north',1)
        if side == 'north':
            offset = 1
            oppo = 'south'
        else:
            offset = -7
            oppo = 'north'
        # if there is only one available move,
        # simply return that move
        if len(available_moves) == 1:
            return  available_moves[0]
        # create a list to record the position_score of each position
        # it takes into account maximise self points
        # minimize opponent points and the availability of self extra move
        position_scores = []
        # i is the index of the position available
        for i in available_moves:
            newboard = deepcopy(board)
            game.board = newboard
            info = game.step(side,i+offset)
            # if the move can lead to win, perform that move
            if info == (side + ':has won'):
                # print("detected a wining move......")
                return i
            # else, record the position score
            position_score = 0
            # if it can give an extra move, it get the point depend on how
            # close it is to the self scoring well
            if info == (side + ':continue'):
                # print("has an extra move")
                position_score += (i+offset)
            # it can also get the score according to how much points it can bring
            position_score += SimpleAgent.points_increase(board,game.board,side)
            # it will get an negative score when its opponent can increase their point
            oppo_available = SimpleAgent.get_available_move(game.board,oppo)
            position_score -= SimpleAgent.max_increase_normal(board,oppo,oppo_available)
            position_scores.append(position_score)
        # print("The position scores")
        # print(position_scores)
        return available_moves[position_scores.index(max(position_scores))]

In [7]:
game = Mancala()
myboard = np.array([0,8,8,8,8,8,8,1,7,7,7,7,7,7,7,0])
game.board = myboard
player = 'south'
print(game)
game.step(player, SimpleAgent.get_move(myboard, player))
print(game)

 1  [  8   8   8   8   8   8   0]
    [  7   7   7   7   7   7   7]  0
Suggest move is: 1
 1  [  8   8   8   8   8   8   0]
    [  0   8   8   8   8   8   8]  1


### Code for parser

In [ ]:
import socket, random, parser

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

print('connecting to localhost 12345')
sock.bind(('localhost', 12345))
sock.listen(1)
server, address = sock.accept()

def receive_line(client):
    buffer = bytearray()
    while True:
        data = client.recv(4)
        buffer.extend(data)
        if b'\n' in data:
            break
    return buffer

def msg_to_board(msg):
    board = [int(str) for str in msg]
    return '{:2d}  {}\n    {}  {}'.format(
        board[7],
        board[0:7][::-1],
        board[8:15],
        board[15]
    )

def msg_parser(msg):
    if msg.startswith('START'): # intialise the board and print it
        print('Received: ' + msg)
        ini_board = np.full(16, 7)
        ini_board[7] = 0
        ini_board[15] = 0
        str_board = msg_to_board(ini_board)
        print(str_board)
        
    # print the board after anyone makes a move
    if msg.endswith('OPP') or msg.endswith('YOU') or (msg.endswith('END') and msg != 'END'): 
        
        change_str, moved_hole, board, side = msg.split(';')
            
        print('YOU' if side == 'OPP' else 'OPP', ' moved ', moved_hole, 'th hole')
        print('The board after the move is:')
        str_board = msg_to_board(board.split(','))
        print(str_board)
        print()

    if msg.startswith('START') or msg.endswith('YOU'): # our turn, make moves
        move = random.randrange(7) + 1
        
        ## make sure the bot will not choose a hole has no stone in it
        if msg.endswith('YOU'):
            while board.split(',')[7+move] == '0':
                move = random.randrange(7) + 1
        
        text = 'MOVE;' + str(move) + '\n'
        server.sendall(text.encode('utf-8'))
        print('Send: ' + str(text))
    
    if msg == 'END':
        print(msg)
        return
        
    
while True:
    text = receive_line(server).decode('utf-8').rstrip()
    msg_parser(text)